In [1]:
# 周玮康 3017207087

def scanData(D, ck, minsupport=0.5):
    dict = {}
    for c in ck:
        for items in D:
            # 集合的函数 可以用来判断是否是子集**
            if c.issubset(items):
                if c in dict:
                    # 字典中可以用set做key **
                    dict[c] += 1
                else:
                    dict[c] = 1
    relist = []
    support = {}  # 每个数据值的支持度
    # 可以for遍历字典得到key值
    for key in dict:
        if dict[key] / len(D) >= minsupport:
            relist.append(key)
        support[key] = round(dict[key] / len(D), 2)
    return relist, support



In [2]:

def apriorigen(DS):
    retlist = []
    lenLk = len(DS)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            if len(DS[i] | DS[j]) - len(DS[i]) == 1:
                if not DS[i] | DS[j] in retlist:
                    retlist.append(DS[i] | DS[j])
    return retlist


In [3]:


# 判断在fre 大集合中， 每个元素组合的关系是否符合conf >= minconf
def conf(fre, H, supportData, brl, minconf=0.7):
    # 用来记录这个元素已经被推断过
    pruneH = []
    for element in H:
        conf = round(supportData[fre] / supportData[fre - element], 2)
        if conf >= minconf:
            print(fre - element, '==>', element, ' :conf = ', conf)
            brl.append([fre - element, fre, conf])
            pruneH.append(element)
    return pruneH

In [4]:
# 用于使用候选规则对关联规则计算做剪枝
# 规则： 若左部为{0，1，2}=>{3}不满足conf 则 左部为{1，2，0}子集的也不会满足 因为子集的支持度增大，即出现的概率增大，而分子概率不变
# 即 conf = $\frac{X U Y}{X}$
def rulesFromConseq(fre, H, supportData, brl, minconf):
    # 获取每个集合的元素个数，即i
    # H为出现在规则右边的集合的列表，判断目前打算用来做子集合中元素的个数
    m = len(H[0])
    # 满足 比子集 大 2 才有合并的必要 ，见5行后的注释详细解释
    if len(fre) > (m + 1):
        # 生成H中元素无重复组合
        Hmp1 = apriorigen(H)
        # e.g. 假如为第一次进入，则每个Hmp1中只有大小为1的子集合，经上步当生成后获得大小为2的子集合
        # 并将一个大小为2的支持度符合的值子集合传回，为下次递归生成 3元素子集合做准备，直到发现子集的个数+2的子集与全集相等停止
        Hmp1 = conf(fre, Hmp1, supportData, brl, minconf)
        if (len(Hmp1) > 1):
            # 递归
            rulesFromConseq(fre, Hmp1, supportData, brl, minconf)


In [5]:
if __name__ == '__main__':
    '''
    总思路：
        首先求单个元素，计算每个元素支持值，判断是否保留
        用单元素集求双元素候选集
        计算每个元素支持值，判断是否保留
        继续计算候选值
        计算支持值
        ……
        直到由于支持值将所有集合内元素抛出
        结束循环
    '''
    dataSet = [['V', 'D', 'C', 'M'],
               ['U', 'D', 'C', 'M', 'N', 'F'],
               ['D', 'A', 'MN', 'G', 'N'],
               ['D', 'U', 'M', 'N', 'F'],
               ['D', 'V', 'M', 'G', 'A'],
               ['U', 'M', 'D', 'I', 'N']]
    # 将数据转化成字典 单转map无法直观看出， 无需遍历，直接list即可
    D = list(map(set, dataSet))

    # 求取无重复的不变集合
    # 使用‘[]’能够保证元素不散 ***很重要
    singleSet = []
    for items in dataSet:
        for item in items:
            if not [item] in singleSet:
                singleSet.append([item])
    print(singleSet)

    C1 = list(map(frozenset, singleSet))

    supportData = {}
    L1, t = scanData(D, C1)
    supportData.update(t)
    L = [L1]
    k = 1
    while (len(L[k - 1]) > 0):
        Ck = apriorigen(L[k - 1])
        Lk, t = scanData(D, Ck)
        supportData.update(t)  # 更新字典（把新出现的集合:支持度加入到supportData中）
        L.append(Lk)
        k += 1

    print(L)
    print(supportData)

    # 求出关联组合 并且每次剪枝
    brl = []
    minconf = 0.7
    # 从二元素集合开始遍历总集合
    for i in range(1, len(L)):
        # 遍历所有二元素集合 fre为每个子选课集合
        for fre in L[i]:
            # 获得每个fre中的所有单个元素
            H = [frozenset([item]) for item in fre]
            if i > 1:
                # 由于fre中元素种类变多，需要做分割,使用候选规则
                rulesFromConseq(fre, H, supportData, brl, minconf)
            else:
                conf(fre, H, supportData, brl, minconf)

    print(brl)


[['V'], ['D'], ['C'], ['M'], ['U'], ['N'], ['F'], ['A'], ['MN'], ['G'], ['I']]
[[frozenset({'D'}), frozenset({'M'}), frozenset({'U'}), frozenset({'N'})], [frozenset({'M', 'D'}), frozenset({'U', 'D'}), frozenset({'N', 'D'}), frozenset({'M', 'U'}), frozenset({'M', 'N'}), frozenset({'U', 'N'})], [frozenset({'M', 'D', 'U'}), frozenset({'M', 'N', 'D'}), frozenset({'U', 'N', 'D'}), frozenset({'M', 'N', 'U'})], [frozenset({'U', 'N', 'M', 'D'})], []]
{frozenset({'V'}): 0.33, frozenset({'D'}): 1.0, frozenset({'C'}): 0.33, frozenset({'M'}): 0.83, frozenset({'U'}): 0.5, frozenset({'N'}): 0.67, frozenset({'F'}): 0.33, frozenset({'A'}): 0.33, frozenset({'MN'}): 0.17, frozenset({'G'}): 0.33, frozenset({'I'}): 0.17, frozenset({'M', 'D'}): 0.83, frozenset({'U', 'D'}): 0.5, frozenset({'N', 'D'}): 0.67, frozenset({'M', 'U'}): 0.5, frozenset({'M', 'N'}): 0.5, frozenset({'U', 'N'}): 0.5, frozenset({'M', 'D', 'U'}): 0.5, frozenset({'M', 'N', 'D'}): 0.5, frozenset({'U', 'N', 'D'}): 0.5, frozenset({'M', 'N',

In [6]:
with open('long_stopwords.txt','r') as f:
    t = f.read()

In [7]:
t.split('\n')

['a',
 'able',
 'about',
 'above',
 'abst',
 'accordance',
 'according',
 'accordingly',
 'across',
 'act',
 'actually',
 'added',
 'adj',
 'after',
 'afterwards',
 'again',
 'against',
 'ah',
 'all',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'am',
 'among',
 'amongst',
 'an',
 'and',
 'announce',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anymore',
 'anyone',
 'anything',
 'anyway',
 'anyways',
 'anywhere',
 'apparently',
 'approximately',
 'are',
 'aren',
 'arent',
 'arise',
 'around',
 'as',
 'aside',
 'ask',
 'asking',
 'at',
 'auth',
 'b',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'begin',
 'beginning',
 'beginnings',
 'begins',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'biol',
 'both',
 'brief',
 'briefly',
 'but',
 'by',
 'c',
 'ca',
 'came',
 'can',
 'cannot',
 "can't",
 'cause',
 'causes',
 'certain',
 'certainly',
 'co',
 'com',
 'come',
 'comes',
 'contain'